In [ ]:
%pip install selenium webdriver-manager

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException


In [82]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

# danh sách team từ URL
team_param = "54"

teams = team_param.split(",")

data = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
for team_id in teams:
    print(f"\n=== Đang lấy dữ liệu cho team {team_id} ===")
    
    url = f"https://www.premierleague.com/en/players?competition=8&season=2025&team={team_id}"
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    # --- Accept cookies ---
    try:
        accept_btn = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_btn.click()
        print("Đã Accept Cookies.")
    except TimeoutException:
        pass

    # --- Close sheet ---
    try:
        close_btn = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sheet__close-btn")))
        driver.execute_script("arguments[0].click();", close_btn)
        print("Đã đóng sheet.")
    except TimeoutException:
        pass

    # --- Thu thập dữ liệu ---
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        players = soup.find_all('tr', class_='player-listings-row')

        for player in players:
            name = player.find('p', class_='player-listings-row__player-name').get_text(strip=True)
            club = player.find('td', class_='player-listings-row__data player-listings-row__data--club').get_text(strip=True)
            position = player.find('td', class_='player-listings-row__data player-listings-row__data--position').get_text(strip=True)
            nation = player.find('td', class_='player-listings-row__data player-listings-row__data--nationality').get_text(strip=True)
            img_tag = soup.find("div", {"data-testid": "playerHeadshot"}).find("img")
            img_url = img_tag["src"] if img_tag else None

            data.append([name, club, position, nation,img_url])
            

        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Next"]')
            if next_button.get_attribute("disabled"):
                print(f"Team {team_id} - đã đến trang cuối.")
                break
            try:
                next_button.click()
            except Exception:
                driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)
        except Exception:
            break

driver.quit()



=== Đang lấy dữ liệu cho team 54 ===
Đã Accept Cookies.
Đã đóng sheet.
Team 54 - đã đến trang cuối.


In [ ]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

team_param = "54"  # Fulham
teams = team_param.split(",")

data = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for team_id in teams:
    print(f"\n=== Đang lấy dữ liệu cho team {team_id} ===")
    
    url = f"https://www.premierleague.com/en/players?competition=8&season=2025&team={team_id}"
    driver.get(url)
    wait = WebDriverWait(driver, 10)

    # Accept cookies
    try:
        accept_btn = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_btn.click()
        print("Đã Accept Cookies.")
    except TimeoutException:
        pass

    # Close sheet
    try:
        close_btn = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sheet__close-btn")))
        driver.execute_script("arguments[0].click();", close_btn)
        print("Đã đóng sheet.")
    except TimeoutException:
        pass

    # Thu thập dữ liệu
    while True:
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        players = soup.find_all('tr', class_='player-listings-row')

        for player in players:
            name = player.find('p', class_='player-listings-row__player-name').get_text(strip=True)
            club = player.find('td', class_='player-listings-row__data--club').get_text(strip=True)
            position = player.find('td', class_='player-listings-row__data--position').get_text(strip=True)
            nation = player.find('td', class_='player-listings-row__data--nationality').get_text(strip=True)
            
            img_tag = player.find("img")
            img_url = img_tag["src"] if img_tag else None

            # Mở trang chi tiết cầu thủ
            a_tag = player.find('a', class_='player-listings-row__player')
            player_url = 'https://www.premierleague.com' + a_tag['href']
            driver.get(player_url)
            time.sleep(2)

            soup = BeautifulSoup(driver.page_source, "html.parser")
            bios = soup.find_all('p', class_='player-profile-bio__item-value')
            if len(bios) >= 2:
                preferred_foot = bios[0].get_text(strip=True)
                dob = bios[1].get_text(strip=True)
            else:
                preferred_foot, dob = None, None

            data.append([name, club, position, nation, img_url, dob, preferred_foot])

            # 🔙 Quay lại danh sách cầu thủ
            driver.back()
            time.sleep(2)

        # Next page
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, 'button[aria-label="Next"]')
            if next_button.get_attribute("disabled"):
                print(f"Team {team_id} - đã đến trang cuối.")
                break
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(2)
        except Exception:
            break

driver.quit()

# Xuất ra DataFrame
df = pd.DataFrame(data, columns=["name", "club", "position", "nation", "img_url", "dob", "preferred_foot"])
print(df.head())



=== Đang lấy dữ liệu cho team 54 ===
Đã Accept Cookies.
Đã đóng sheet.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import time, re

all_html = []

for i in range(3):  # MW1 → MW4
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get("https://www.premierleague.com/fixtures")
    wait = WebDriverWait(driver, 3)

    try:
        accept_btn = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_btn.click()
    except TimeoutException:
        pass

    # Mở filter Matchweek
    filter_btns = driver.find_elements(By.CSS_SELECTOR, 'button[aria-label="Filter By: "]')
    for btn in filter_btns:
        label = btn.find_element(By.CLASS_NAME, "chip__label").text.strip()
        if "Matchweek" in label:
            driver.execute_script("arguments[0].click();", btn)
            break

    
    
    close_btn = wait.until(
    	EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Close Sheet']"))
	)
    driver.execute_script("arguments[0].click();", close_btn)
    filter_btn = wait.until(
		EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Filter By: '][.//span[contains(text(),'MW') or contains(text(),'Matchweek')]]"))
	)
    driver.execute_script("arguments[0].click();", filter_btn)
    mw_label = wait.until(
    EC.element_to_be_clickable((By.XPATH, "//label[@for='fixtures_matchweek_0']"))
	)
    driver.execute_script("arguments[0].click();", mw_label)

		
    # Save
    save_btn = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[.//span[text()='Save']]")))
    driver.execute_script("arguments[0].click();", save_btn)

    time.sleep(2)  # đợi load dữ liệu
    html = driver.page_source
    all_html.append(html)

    print(f"Đã lấy Matchweek {i+1}")
    driver.quit()


In [198]:
from bs4 import BeautifulSoup

data = []

for i in range(len(all_html)):
	print(f"\n--- Xử lý Matchweek {i+1} ---")
	soup = BeautifulSoup(all_html[i], "html.parser")  # ví dụ MW2
	dates = soup.find_all('div', class_='match-list__day-matches')

	for date in dates:
		day = date.find('span', class_='match-list__day-date').get_text(strip=True)

		matches = date.find_all("a", {"data-testid": "matchCard"})
	
		for match in matches:
			teams = match.find_all('span', class_='match-card__team-name match-card__team-name--short')
			home = teams[0].get_text(strip=True)
			away = teams[1].get_text(strip=True)
			score = match.find('span', class_='match-card__score-label').get_text(strip=True)
			link = match['href']
			id = link.split('/')[-2]
			# lưu vào data
			data.append({
				"id": id,
				"date": day,
				"home": home,
				"away": away,
				"score": score,
				"link": 'https://www.premierleague.com'+link
			})


--- Xử lý Matchweek 1 ---

--- Xử lý Matchweek 2 ---

--- Xử lý Matchweek 3 ---


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def safe_get_text(soup, tag, attrs=None, **kwargs):
    try:
        if attrs:
            el = soup.find(tag, attrs, **kwargs)
        else:
            el = soup.find(tag, **kwargs)
        return el.get_text(strip=True) if el else None
    except Exception:
        return None

data = []
for link in df['link'][:1]:  # test 1 link
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(link+'?tab=stats')
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    driver.quit()
    stats = soup.find_all('div', class_='match-stats__table-row')
    stat_list = []
    for stat in stats:
        stat = stat.get_text(strip=True)
        stat_list.append(stat)
    data.append({
        "match_id": link.split("/")[-2],
        "home_team": safe_get_text(soup, "ul", {"data-testid": "homeTeamGoals"}),
        "away_team": safe_get_text(soup, "ul", {"data-testid": "awayTeamGoals"}),
        "home_yellow_card": safe_get_text(soup, "ul", {"data-testid": "homeTeamYellowCards"}),
        "away_yellow_card": safe_get_text(soup, "ul", {"data-testid": "awayTeamYellowCards"}),
        "home_red_card": safe_get_text(soup, "ul", {"data-testid": "homeTeamRedCards"}),
        "away_red_card": safe_get_text(soup, "ul", {"data-testid": "awayTeamRedCards"}),
        "motm": safe_get_text(soup, "h1", class_="motm__player-name"),
        "stats": stat_list
    })


In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

data = []
for link in df['link'][:1]:  # test 1 link
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(link + '?tab=match+info')

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    driver.quit()

    # lấy tất cả value trong match info
    stat_list = [s.get_text(strip=True) for s in soup.find_all('span', class_='match-details__value')]

    data.append({
        "match_id": link.split("/")[-2],
        "stats": stats
    })


In [28]:
import pandas as pd
data = []
df = pd.read_csv('matches.csv')

In [ ]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# tab stats
driver.get(df['link'][0] + "?tab=stats")
soup_stats = BeautifulSoup(driver.page_source, "html.parser")

stat_texts = [s.get_text(strip=True) for s in soup_stats.find_all('div', class_='match-stats__table-row')]


In [21]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

team_param = "54,3,7,91,94,36,90,8,31,11,2,14,43,1,4,17,56,6,21,39"
teams = team_param.split(",")

data = []

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for team_id in teams:
    url = f"https://www.premierleague.com/en/clubs/{team_id}/overview"
    print("Fetching:", url)
    driver.get(url)

    try:
        # đợi tên đội xuất hiện (tối đa 10 giây)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h1.club-profile-header__title"))
        )

        soup = BeautifulSoup(driver.page_source, "html.parser")

        name = soup.select_one("h1.club-profile-header__title").get_text(strip=True)
        link = soup.select_one("div.analytics-link a")["href"]
        info = soup.select("p.club-profile-bio__metadata-value")
        est = info[0].get_text(strip=True) if len(info) > 0 else None
        stadium = info[1].get_text(strip=True) if len(info) > 1 else None

        data.append({
            "team_id": team_id,
            "name": name,
            "link": link,
            "est": est,
            "stadium": stadium
        })

    except Exception as e:
        print(f"❌ Lỗi với team_id={team_id}: {e}")

driver.quit()

print(data)


Fetching: https://www.premierleague.com/en/clubs/54/overview
Fetching: https://www.premierleague.com/en/clubs/3/overview
Fetching: https://www.premierleague.com/en/clubs/7/overview
Fetching: https://www.premierleague.com/en/clubs/91/overview
Fetching: https://www.premierleague.com/en/clubs/94/overview
Fetching: https://www.premierleague.com/en/clubs/36/overview
Fetching: https://www.premierleague.com/en/clubs/90/overview
Fetching: https://www.premierleague.com/en/clubs/8/overview
Fetching: https://www.premierleague.com/en/clubs/31/overview
Fetching: https://www.premierleague.com/en/clubs/11/overview
Fetching: https://www.premierleague.com/en/clubs/2/overview
Fetching: https://www.premierleague.com/en/clubs/14/overview
Fetching: https://www.premierleague.com/en/clubs/43/overview
Fetching: https://www.premierleague.com/en/clubs/1/overview
Fetching: https://www.premierleague.com/en/clubs/4/overview
Fetching: https://www.premierleague.com/en/clubs/17/overview
Fetching: https://www.premierl

In [25]:
df = pd.DataFrame(data, columns=["team_id", "name", "link", "est", "stadium"])
df.head()	
df.to_csv("teams.csv", index=False)

In [ ]:
df.shape[0]

(15, 3)

In [2]:
import pandas as pd
matches = pd.read_csv('E:\ecommerce-pipeline\data\matches.csv')
matches.head()

,id,date,home,away,score,link
0,2561895,Sat 16 Aug,Liverpool,Bournemouth,4 - 2,https://www.premierleague.com/en/match/2561895...
1,2561896,Sat 16 Aug,Aston Villa,Newcastle,0 - 0,https://www.premierleague.com/en/match/2561896...
2,2561897,Sat 16 Aug,Brighton,Fulham,1 - 1,https://www.premierleague.com/en/match/2561897...
3,2561899,Sat 16 Aug,Sunderland,West Ham,3 - 0,https://www.premierleague.com/en/match/2561899...
4,2561900,Sat 16 Aug,Spurs,Burnley,3 - 0,https://www.premierleague.com/en/match/2561900...


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import json

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
all_matches = []

for i, link in enumerate(matches['link']):
    prefix = link
    data = {
        "home": {
            "name":None, 
            "goals": [], "yellow_cards": [], "red_cards": []},
        "away": {"goals": [], "yellow_cards": [], "red_cards": []},
        "details": {},
        "motm": None
    }

    # --- Goals ---
    driver.get(f"{prefix}?tab=stats")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    for goal in soup.find_all('ul', {"data-testid": "homeTeamGoals"}):
        for g in goal.find_all('li', {"data-testid": 'scoreboardGoalEvent'}):
            data["home"]["goals"].append(g.get_text(strip=True))

    for goal in soup.find_all('ul', {"data-testid": "awayTeamGoals"}):
        for g in goal.find_all('li', {"data-testid": 'scoreboardGoalEvent'}):
            data["away"]["goals"].append(g.get_text(strip=True))

    # --- Yellow Cards ---
    for card in soup.find_all('ul', {"data-testid": "homeTeamYellowCards"}):
        for c in card.find_all('li', {"data-testid": 'scoreboardCardEvent'}):
            data["home"]["yellow_cards"].append(c.get_text(strip=True))

    for card in soup.find_all('ul', {"data-testid": "awayTeamYellowCards"}):
        for c in card.find_all('li', {"data-testid": 'scoreboardCardEvent'}):
            data["away"]["yellow_cards"].append(c.get_text(strip=True))

    # --- Stats ---
    driver.get(f"{prefix}?tab=stats")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    home_stats = soup.find_all(
        "div", class_=re.compile(r"match-stats__table-cell.*--home.*stat-value")
    )
    away_stats = soup.find_all(
        "div", class_=re.compile(r"match-stats__table-cell.*--away.*stat-value")
    )
    infos = soup.find_all('div', class_='match-stats__table-cell match-stats__stat-name')

    for stat_h, stat_a, info in zip(home_stats, away_stats, infos):
        key = info.get_text(strip=True)
        data['home'][key] = stat_h.get_text(strip=True)
        data['away'][key] = stat_a.get_text(strip=True)

    # --- Match Info ---
    driver.get(f"{prefix}?tab=match+info")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    infos = soup.find_all('div', {'data-testid': 'matchDetailsEntry'})
    for info in infos:
        key = info.find('span', class_='match-details__key').get_text(strip=True)
        value = info.find('span', class_='match-details__value').get_text(strip=True)
        data["details"][key] = value

    # --- MOTM ---
    driver.get(f"{prefix}?tab=recap")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    motm_fn = soup.find('span', class_='motm__player-name--first-name')
    motm_ln = soup.find('span', class_='motm__player-name--last-name')
    if motm_fn and motm_ln:
        data['motm'] = f"{motm_fn.get_text(strip=True)} {motm_ln.get_text(strip=True)}"

    # Lưu ngay sau mỗi link
    all_matches.append(data)
    with open("matches.json", "w", encoding="utf-8") as f:
        json.dump(all_matches, f, ensure_ascii=False, indent=2)

driver.quit()


In [4]:
df = pd.read_json('matches.json')

In [ ]:
import pandas as pd

def calculate_week(date):
    start_date = pd.to_datetime('16-08-2025', format='%d-%m-%Y')
    date = pd.to_datetime(date, format='%d-%m-%Y')
    diff_date = (date - start_date).days
    
    nguyen = diff_date //7 
    du = diff_date %7
    return nguyen+1,du
    
    

In [101]:
calculate_week('13-09-2025')

(5, 0)

In [15]:
import pandas as pd


df = pd.read_parquet(r"C:\Users\thaia\Downloads\teams_dataset (1).parquet")
df

,team_id,name,link,est,stadium
0,54,Fulham,https://www.fulhamfc.com/,1879,Craven Cottage
1,3,Arsenal,https://www.arsenal.com/,1886,Emirates Stadium
2,7,Aston Villa,https://www.avfc.co.uk/,1874,Villa Park
3,91,Bournemouth,https://www.afcb.co.uk/,1890,Vitality Stadium
4,94,Brentford,https://www.brentfordfc.com/en,1889,Gtech Community Stadium
5,36,Brighton and Hove Albion,https://www.brightonandhovealbion.com/pages/en...,1901,American Express Stadium
6,90,Burnley,https://www.burnleyfootballclub.com/,1882,Turf Moor
7,8,Chelsea,https://www.chelseafc.com/en,1905,Stamford Bridge
8,31,Crystal Palace,https://www.cpfc.co.uk/,1861,Selhurst Park
9,11,Everton,https://www.evertonfc.com/,1878,Hill Dickinson Stadium


In [10]:
stadium = df.copy()
teams = pd.read_csv('teams.csv')
stadium.head()


,Stadium,City,HomeTeams,Capacity,Country,IOC
0,Stadiumi Besëlidhja,Lezhë,Besëlidhja,7000,Albania,ALB
1,Stadiumi Flamurtari,Vlorë,Flamurtari Vlorë,8200,Albania,ALB
2,Stadiumi Laçi,Laçi,KF Laçi,5000,Albania,ALB
3,Stadiumi Niko Dovana,Durrës,Teuta,12040,Albania,ALB
4,Stadiumi Selman Stërmasi,Tirana,"KF Tirana, Dinamo, Partizani",9500,Albania,ALB


In [12]:
# teams có cột 'stadium_id', stadium có cột 'id'
pd.merge(teams, stadium, how='left', left_on='stadium', right_on='Stadium')


,team_id,name,link,est,stadium,Stadium,City,HomeTeams,Capacity,Country,IOC
0,54,Fulham,https://www.fulhamfc.com/,1879,Craven Cottage,Craven Cottage,London,Fulham,25700.0,England,ENG
1,3,Arsenal,https://www.arsenal.com/,1886,Emirates Stadium,Emirates Stadium,London,Arsenal,59867.0,England,ENG
2,7,Aston Villa,https://www.avfc.co.uk/,1874,Villa Park,Villa Park,Birmingham,Aston Villa,42682.0,England,ENG
3,91,Bournemouth,https://www.afcb.co.uk/,1890,Vitality Stadium,Vitality Stadium,Bournemouth,AFC Bournemouth,11360.0,England,ENG
4,94,Brentford,https://www.brentfordfc.com/en,1889,Gtech Community Stadium,NaN,NaN,NaN,NaN,NaN,NaN
5,36,Brighton and Hove Albion,https://www.brightonandhovealbion.com/pages/en...,1901,American Express Stadium,NaN,NaN,NaN,NaN,NaN,NaN
6,90,Burnley,https://www.burnleyfootballclub.com/,1882,Turf Moor,Turf Moor,Burnley,Burnley,21944.0,England,ENG
7,8,Chelsea,https://www.chelseafc.com/en,1905,Stamford Bridge,Stamford Bridge,London,Chelsea,41631.0,England,ENG
8,31,Crystal Palace,https://www.cpfc.co.uk/,1861,Selhurst Park,Selhurst Park,London,Crystal Palace,25456.0,England,ENG
9,11,Everton,https://www.evertonfc.com/,1878,Hill Dickinson Stadium,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
matches = pd.read_csv('matches.csv')

In [4]:
import time
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

detailed_info = matches[['link']].copy()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 10)  # chờ tối đa 10s

all_matches = []

for i, link in enumerate(detailed_info["link"][:1]):

    prefix = link
    data = {
        "match_id": link.split('/')[-2],
        "home": {"goals": [],'assists': [], "yellow_cards": [], "red_cards": []},
        "away": {"goals": [],'assists': [], "yellow_cards": [], "red_cards": []},
        "motm": None
    }

    # --- Goals + Yellow Cards + Stats ---
    driver.get(f"{prefix}?tab=stats")
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "ul[data-testid='homeTeamGoals']")))
    soup = BeautifulSoup(driver.page_source, "html.parser")

    for goal in soup.find_all('ul', {"data-testid": "homeTeamGoals"}):
        for g in goal.find_all('span', {"data-testid": 'scoreboardEventScorer'}):
            data["home"]["goals"].append(g.get_text(strip=True))
        for g in goal.find_all('span', {"data-testid": 'scoreboardEventAssists'}):
            data["home"]["assists"].append(g.get_text(strip=True))

    for goal in soup.find_all('ul', {"data-testid": "awayTeamGoals"}):
        for g in goal.find_all('span', {"data-testid": 'scoreboardEventScorer'}):
            data["away"]["goals"].append(g.get_text(strip=True))
        for g in goal.find_all('span', {"data-testid": 'scoreboardEventAssists'}):
            data["away"]["assists"].append(g.get_text(strip=True))

    for card in soup.find_all('ul', {"data-testid": "homeTeamYellowCards"}):
        for c in card.find_all('li', {"data-testid": 'scoreboardCardEvent'}):
            data["home"]["yellow_cards"].append(c.get_text(strip=True))

    for card in soup.find_all('ul', {"data-testid": "awayTeamYellowCards"}):
        for c in card.find_all('li', {"data-testid": 'scoreboardCardEvent'}):
            data["away"]["yellow_cards"].append(c.get_text(strip=True))

    home_stats = soup.find_all(
        "div", class_=re.compile(r"match-stats__table-cell.*--home.*stat-value")
    )
    away_stats = soup.find_all(
        "div", class_=re.compile(r"match-stats__table-cell.*--away.*stat-value")
    )
    infos = soup.find_all('div', class_='match-stats__table-cell match-stats__stat-name')

    for stat_h, stat_a, info in zip(home_stats, away_stats, infos):
        key = info.get_text(strip=True)
        data['home'][key] = stat_h.get_text(strip=True)
        data['away'][key] = stat_a.get_text(strip=True)

    # --- Match Info ---
    driver.get(f"{prefix}?tab=match+info")
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='matchDetailsEntry']")))
    soup = BeautifulSoup(driver.page_source, "html.parser")

    infos = soup.find_all('div', {'data-testid': 'matchDetailsEntry'})
    for info in infos:
        key = info.find('span', class_='match-details__key').get_text(strip=True)
        value = info.find('span', class_='match-details__value').get_text(strip=True)
        data[key] = value

    # --- MOTM ---
    driver.get(f"{prefix}?tab=recap")
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.motm__player-name--first-name")))
    soup = BeautifulSoup(driver.page_source, "html.parser")

    motm_fn = soup.find('span', class_='motm__player-name--first-name')
    motm_ln = soup.find('span', class_='motm__player-name--last-name')
    if motm_fn and motm_ln:
        data['motm'] = f"{motm_fn.get_text(strip=True)} {motm_ln.get_text(strip=True)}"

    all_matches.append(data)

driver.quit()

result_df = pd.json_normalize(all_matches)


In [73]:
import pandas as pd

df = pd.read_parquet(r"C:\Users\thaia\Downloads\week_1 (8).parquet")

# df[["player_name", "minute"]] = df["yellow_card"].str.extract(r"(.+?)\s+(\d+)'")


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time

team_param = "54,3,7,91,94,36,90,8,31,11,2,14,43,1,4,17,56,6,21,39"
teams = team_param.split(",")

data = []
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 10)  # ⏳ chờ tối đa 10s cho mỗi element
output_file = "players.csv"

# Tạo file CSV rỗng ban đầu với header
pd.DataFrame(columns=["player_id","name", "club", "position", "nation", "img_url", "dob", "preferred_foot", "number"])\
    .to_csv(output_file, index=False)

for team_id in teams:
    print(f"\n=== Đang lấy dữ liệu cho team {team_id} ===")

    url = f"https://www.premierleague.com/en/players?competition=8&season=2025&team={team_id}"
    driver.get(url)

    # Accept cookies
    try:
        accept_btn = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_btn.click()
        print("Đã Accept Cookies.")
    except TimeoutException:
        pass

    # Close sheet (nếu có)
    try:
        close_btn = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sheet__close-btn")))
        driver.execute_script("arguments[0].click();", close_btn)
        print("Đã đóng sheet.")
    except TimeoutException:
        pass

    # Thu thập dữ liệu
    while True:
        # ✅ Chờ danh sách cầu thủ load
        try:
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "player-listings-row")))
        except TimeoutException:
            print("⚠️ Không load được danh sách cầu thủ.")
            break

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        players = soup.find_all('tr', class_='player-listings-row')

        for player in players:
            name = player.find('p', class_='player-listings-row__player-name').get_text(strip=True)
            club = player.find('td', class_='player-listings-row__data--club').get_text(strip=True)
            position = player.find('td', class_='player-listings-row__data--position').get_text(strip=True)
            nation = player.find('td', class_='player-listings-row__data--nationality').get_text(strip=True)

            # Xử lý ảnh
            img_tag = player.find("img")
            img_url = img_tag["src"] if img_tag else None
            lists = img_url.split('/')
            lists[-2] = '110x140'   # chỉnh lại kích thước
            new_url = '/'.join(lists)

            # Mở trang chi tiết cầu thủ
            a_tag = player.find('a', class_='player-listings-row__player')
            player_url = 'https://www.premierleague.com' + a_tag['href']
            driver.get(player_url)

            try:
                wait.until(EC.presence_of_element_located((By.CLASS_NAME, "player-profile-header__shirt-number")))
            except TimeoutException:
                print(f"⚠️ Không load kịp dữ liệu cho {name}")
                driver.back()
                continue

            soup = BeautifulSoup(driver.page_source, "html.parser")

            # Lấy số áo
            number_tag = soup.find('p', class_='player-profile-header__shirt-number')
            number = number_tag.get_text(strip=True) if number_tag else None

            # Lấy foot + dob
            bios = soup.find_all('p', class_='player-profile-bio__item-value')
            if len(bios) >= 2:
                preferred_foot = bios[0].get_text(strip=True)
                dob = bios[1].get_text(strip=True)
            else:
                preferred_foot, dob = None, None
            player_id = img_url.split('.')[2].split('/')[-1]

            data_row = [player_id, name, club, position, nation, new_url, dob, preferred_foot, number]

            # ✅ Lưu trực tiếp vào CSV ngay sau khi lấy được
            pd.DataFrame([data_row]).to_csv(
                output_file, mode="a", header=False, index=False, encoding="utf-8"
            )
            print(data_row)
            driver.back()
            time.sleep(3)

        # Next page
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]')))
            if next_button.get_attribute("disabled"):
                print(f"Team {team_id} - đã đến trang cuối.")
                break
            driver.execute_script("arguments[0].click();", next_button)
        except TimeoutException:
            break

driver.quit()

# Xuất ra DataFrame
# df = pd.DataFrame(data, columns=["name", "club", "position", "nation", "img_url", "dob", "preferred_foot", "number"])
# print(df.head())


In [2]:
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import threading

output_file = "players.csv"
lock = threading.Lock()

# Hàm crawl cho 1 team
def crawl_team(team_id):
    from bs4 import BeautifulSoup
    from selenium.webdriver.common.by import By
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.common.exceptions import TimeoutException
    import time

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    wait = WebDriverWait(driver, 10)

    url = f"https://www.premierleague.com/en/players?competition=8&season=2025&team={team_id}"
    driver.get(url)

    try:
        accept_btn = wait.until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler")))
        accept_btn.click()
    except TimeoutException:
        pass

    results = []
    while True:
        try:
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "player-listings-row")))
        except TimeoutException:
            break

        soup = BeautifulSoup(driver.page_source, "html.parser")
        players = soup.find_all('tr', class_='player-listings-row')

        for player in players:
            name = player.find('p', class_='player-listings-row__player-name').get_text(strip=True)
            club = player.find('td', class_='player-listings-row__data--club').get_text(strip=True)
            position = player.find('td', class_='player-listings-row__data--position').get_text(strip=True)
            nation = player.find('td', class_='player-listings-row__data--nationality').get_text(strip=True)

            img_tag = player.find("img")
            img_url = img_tag["src"] if img_tag else None
            lists = img_url.split('/')
            lists[-2] = '110x140'
            new_url = '/'.join(lists)

            results.append([team_id, name, club, position, nation, new_url])

        # Next page
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Next"]')))
            if next_button.get_attribute("disabled"):
                break
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(1)
        except TimeoutException:
            break

    driver.quit()

    # Ghi dữ liệu vào CSV (có lock để tránh ghi đè)
    with lock:
        pd.DataFrame(results, columns=["team_id", "name", "club", "position", "nation", "img_url"])\
            .to_csv(output_file, mode="a", header=False, index=False, encoding="utf-8")

    return f"Team {team_id} xong {len(results)} cầu thủ."

# ----------------------------
if __name__ == "__main__":
    team_param = "54,3,7,91,94,36,90,8,31,11,2,14,43,1,4,17,56,6,21,39"
    teams = team_param.split(",")

    # Tạo file CSV với header trước
    pd.DataFrame(columns=["team_id","name","club","position","nation","img_url"])\
        .to_csv(output_file, index=False)

    with ThreadPoolExecutor(max_workers=5) as executor:  # chạy song song 5 team
        futures = [executor.submit(crawl_team, t) for t in teams]
        for f in as_completed(futures):
            print(f.result())


Team 54 xong 31 cầu thủ.
Team 94 xong 45 cầu thủ.
Team 3 xong 49 cầu thủ.
Team 91 xong 46 cầu thủ.
Team 7 xong 53 cầu thủ.
Team 11 xong 49 cầu thủ.
Team 8 xong 59 cầu thủ.
Team 31 xong 49 cầu thủ.
Team 36 xong 49 cầu thủ.
Team 90 xong 52 cầu thủ.
Team 2 xong 43 cầu thủ.
Team 43 xong 47 cầu thủ.
Team 14 xong 54 cầu thủ.
Team 1 xong 57 cầu thủ.
Team 17 xong 50 cầu thủ.
Team 4 xong 46 cầu thủ.
Team 6 xong 64 cầu thủ.
Team 56 xong 56 cầu thủ.
Team 21 xong 48 cầu thủ.
Team 39 xong 46 cầu thủ.


In [6]:
import pandas as pd

pd.read_parquet(r"C:\Users\thaia\Downloads\player_info (1).parquet").tail()

,team_id,name,club,position,nation,img_url
988,39,Jørgen Strand Larsen,Wolverhampton Wanderers,Forward,Norway,https://resources.premierleague.com/premierlea...
989,39,Chiquinho,Wolverhampton Wanderers,Forward,Portugal,https://resources.premierleague.com/premierlea...
990,39,Jackson Tchatchoua,Wolverhampton Wanderers,Defender,Cameroon,https://resources.premierleague.com/premierlea...
991,39,Boubacar Traoré,Wolverhampton Wanderers,Midfielder,Mali,https://resources.premierleague.com/premierlea...
992,39,André,Wolverhampton Wanderers,Midfielder,Brazil,https://resources.premierleague.com/premierlea...


In [20]:
import time

In [28]:
from selenium.webdriver.support.ui import WebDriverWait


In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://www.premierleague.com/en/match/2561940/liverpool-vs-everton?tab=lineups')

wait = WebDriverWait(driver,10)
html = driver.page_source
driver.quit()

In [25]:
from bs4 import BeautifulSoup
import json

soup = BeautifulSoup(html, "html.parser")

# --- Đội hình xuất phát ---
teams = soup.find_all("div", {"data-testid": "lineupsTeamFormation"})

data = {"home": {"starting": [], "subs": []},
        "away": {"starting": [], "subs": []}}

for idx, team in enumerate(teams):
    players = team.find_all("div", class_="lineups-player__headshot")
    for p in players:
        img = p.find("img")
        if img:
            player_id = img["src"].split('/')[-1].split('.')[0]
            if idx == 0:
                data["home"]["starting"].append(player_id)
            else:
                data["away"]["starting"].append(player_id)

# --- Cầu thủ dự bị vào sân ---
sub_players = soup.find_all("ul", {"data-testid": "squadList"})
for idx, team in enumerate(sub_players):
    players = team.find_all('a', class_="squad-list__item-link")
    for player in players:
        if player.find('div', {"data-testid": "lineupsPlayerSubOnBadge"}) is not None:
            img = player.find("img")
            if img:
                player_id = img["src"].split('/')[-1].split('.')[0]
                if idx == 0:
                    data["home"]["subs"].append(player_id)
                else:
                    data["away"]["subs"].append(player_id)

# Xuất ra JSON
print(json.dumps(data, indent=2, ensure_ascii=False))


{
  "home": {
    "starting": [
      "116535",
      "492777",
      "204716",
      "97032",
      "544877",
      "441266",
      "243016",
      "118748",
      "424876",
      "243298",
      "510663"
    ],
    "subs": [
      "494595",
      "219168",
      "206915"
    ]
  },
  "away": {
    "starting": [
      "111234",
      "512462",
      "17761",
      "106611",
      "224967",
      "80801",
      "232928",
      "440993",
      "215413",
      "114283",
      "486385"
    ],
    "subs": [
      "586309",
      "496661",
      "502697"
    ]
  }
}


In [ ]:
sub_players = soup.find_all('ul',{'data-testid': 'squadList'})
for team in sub_players:
    players = team.find_all('a',class_ = 'squad-list__item-link')
    for player in players:
        if player.find('div',{"data-testid": "lineupsPlayerSubOnBadge"}) != None:
            img = player.find('img')
            #name = player.find('span',class_ = 'squad-list__player-name').get_text(strip=True)
            player_id = img["src"].split('/')[-1].split('.')[0]
            print(player_id)
    print('-----------')

494595
219168
206915
-----------
586309
496661
502697
-----------


In [5]:
import pandas as pd

matches = pd.read_csv('matches.csv')

In [8]:
matches['link'][0]

'https://www.premierleague.com/en/match/2561895/liverpool-vs-bournemouth'

In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

# --- Chuẩn bị dữ liệu và driver ---
detailed_info = matches[['link']].copy()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
wait = WebDriverWait(driver, 10)  # chờ tối đa 10s

all_matches = []

for home, away, link in zip(matches['home'], matches['away'], detailed_info["link"]):
    prefix = link
    data = {
        "match_id": link.split('/')[-2],
        "home": {
            "name": home,
            "goals": [], "assists": [], "yellow_cards": [], "red_cards": [],
            "starting": [], "subs": []
        },
        "away": {
            "name": away,
            "goals": [], "assists": [], "yellow_cards": [], "red_cards": [],
            "starting": [], "subs": []
        },
        "motm": None
    }

    # --- Goals, Assists, Yellow Cards ---
    try:
        driver.get(f"{prefix}?tab=stats")
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # home goals & assists
        for goal in soup.find_all('ul', {"data-testid": "homeTeamGoals"}):
            for g in goal.find_all('span', {"data-testid": 'scoreboardEventScorer'}):
                data["home"]["goals"].append(g.get_text(strip=True))
            for g in goal.find_all('span', {"data-testid": 'scoreboardEventAssists'}):
                data["home"]["assists"].append(g.get_text(strip=True))

        # away goals & assists
        for goal in soup.find_all('ul', {"data-testid": "awayTeamGoals"}):
            for g in goal.find_all('span', {"data-testid": 'scoreboardEventScorer'}):
                data["away"]["goals"].append(g.get_text(strip=True))
            for g in goal.find_all('span', {"data-testid": 'scoreboardEventAssists'}):
                data["away"]["assists"].append(g.get_text(strip=True))

        # yellow cards
        for card in soup.find_all('ul', {"data-testid": "homeTeamYellowCards"}):
            for c in card.find_all('li', {"data-testid": 'scoreboardCardEvent'}):
                data["home"]["yellow_cards"].append(c.get_text(strip=True))
        for card in soup.find_all('ul', {"data-testid": "awayTeamYellowCards"}):
            for c in card.find_all('li', {"data-testid": 'scoreboardCardEvent'}):
                data["away"]["yellow_cards"].append(c.get_text(strip=True))

        # match stats
        home_stats = soup.find_all(
            "div", class_=re.compile(r"match-stats__table-cell.*--home.*stat-value")
        )
        away_stats = soup.find_all(
            "div", class_=re.compile(r"match-stats__table-cell.*--away.*stat-value")
        )
        infos = soup.find_all('div', class_='match-stats__table-cell match-stats__stat-name')

        for stat_h, stat_a, info in zip(home_stats, away_stats, infos):
            key = info.get_text(strip=True)
            data['home'][key] = stat_h.get_text(strip=True)
            data['away'][key] = stat_a.get_text(strip=True)

    except Exception as e:
        print("Stats parse error:", e)

    # --- Match Info ---
    try:
        driver.get(f"{prefix}?tab=match+info")
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-testid='matchDetailsEntry']")))
        soup = BeautifulSoup(driver.page_source, "html.parser")

        infos = soup.find_all('div', {'data-testid': 'matchDetailsEntry'})
        for info in infos:
            key = info.find('span', class_='match-details__key').get_text(strip=True)
            value = info.find('span', class_='match-details__value').get_text(strip=True)
            data[key] = value
    except Exception as e:
        print("Match info parse error:", e)

    # --- Lineups (Starting + Subs) ---
    try:
        driver.get(f"{prefix}?tab=lineups")
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # đội hình xuất phát
        teams = soup.find_all("div", {"data-testid": "lineupsTeamFormation"})
        for idx, team in enumerate(teams):
            players = team.find_all("div", class_="lineups-player__headshot")
            for p in players:
                img = p.find("img")
                if img:
                    player_id = img["src"].split('/')[-1].split('.')[0]
                    player_obj = {"id": player_id, "img": img["src"]}
                    if idx == 0:
                        data["home"]["starting"].append(player_obj)
                    else:
                        data["away"]["starting"].append(player_obj)

        # cầu thủ dự bị có vào sân
        sub_players = soup.find_all("ul", {"data-testid": "squadList"})
        for idx, team in enumerate(sub_players):
            players = team.find_all('a', class_="squad-list__item-link")
            for player in players:
                if player.find('div', {"data-testid": "lineupsPlayerSubOnBadge"}) is not None:
                    img = player.find("img")
                    if img:
                        player_id = img["src"].split('/')[-1].split('.')[0]
                        player_obj = {"id": player_id, "img": img["src"]}
                        if idx == 0:
                            data["home"]["subs"].append(player_obj)
                        else:
                            data["away"]["subs"].append(player_obj)

    except Exception as e:
        print("Lineups parse error:", e)

    # --- MOTM ---
    try:
        driver.get(f"{prefix}?tab=recap")
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.motm__player-name--first-name")))
        soup = BeautifulSoup(driver.page_source, "html.parser")

        motm_fn = soup.find('span', class_='motm__player-name--first-name')
        motm_ln = soup.find('span', class_='motm__player-name--last-name')
        motm_img = soup.find('div', {"data-testid": "playerHeadshot"}).find("img")

       
        data['motm'] = motm_img["src"].split('/')[-1].split('.')[0]

    except Exception as e:
        print("MOTM parse error:", e)

    all_matches.append(data)
    

driver.quit()

# --- Xuất ra DataFrame ---
result_df = pd.json_normalize(all_matches)


In [34]:
result_df

,match_id,motm,Kickoff,Stadium,Attendance,Referee,Assistant Referee #1,Assistant Referee #2,Fourth Official,VAR,...,away.Clearances,away.Total Dribbles,away.Successful Dribbles,away.Duels Won,away.Aerial Duels Won,away.Distance Covered,away.Red Cards,away.Yellow Cards,away.Fouls,away.Offsides
0,2561895,510663,"Sat 16 August 2025, 02:00","Anfield, Liverpool","60,315",Anthony Taylor,Gary Beswick,Craig Taylor,Farai Hallam,Michael Oliver,...,45,19,8,52,18,110.19km,0,2,10,2


In [25]:
import pandas as pd

# đọc parquet
df2 = pd.read_parquet(r"C:\Users\thaia\Downloads\week_1 (3).parquet")

df2

,player_id,first_name,player_last_name,club,position,nation,img_url,match_id,minute,category
0,502697,Charly,Alcaraz,Everton,Midfielder,Argentina,https://resources.premierleague.com/premierlea...,2561904,63',1
1,232892,Calvin,Bassey,Fulham,Defender,Nigeria,https://resources.premierleague.com/premierlea...,2561897,31',1
2,111317,David,Brooks,Bournemouth,Midfielder,Wales,https://resources.premierleague.com/premierlea...,2561895,14',1
3,76357,Tom,Cairney,Fulham,Midfielder,Scotland,https://resources.premierleague.com/premierlea...,2561897,90'+3',1
4,466075,Riccardo,Calafiori,Arsenal,Defender,Italy,https://resources.premierleague.com/premierlea...,2561903,42',1
5,432830,Nathan,Collins,Brentford,Defender,Ireland,https://resources.premierleague.com/premierlea...,2561898,52',1
6,87835,Matt,Doherty,Wolverhampton Wanderers,Defender,Ireland,https://resources.premierleague.com/premierlea...,2561901,67',1
7,596777,Patrick,Dorgu,Manchester United,Defender,Denmark,https://resources.premierleague.com/premierlea...,2561903,85',1
8,624773,,Estêvão,Chelsea,Forward,Brazil,https://resources.premierleague.com/premierlea...,2561902,74',1
9,444102,,Evanilson,Bournemouth,Forward,Brazil,https://resources.premierleague.com/premierlea...,2561895,81',1


In [21]:
df3.to_csv('stadium.csv',index = False)

In [18]:
df2['team_id'] = df2['team_id'].astype(int)


In [19]:
df = pd.merge(df2, df3, how='inner', on='team_id')
df

,team_id,name,link,est,stadium_x,stadium_y,capacity,image_link,location
0,54,Fulham,https://www.fulhamfc.com/,1879,Craven Cottage,Craven Cottage,28.800,https://upload.wikimedia.org/wikipedia/commons...,"Stevenage Road, Fulham, London SW6 6HH, UK"
1,3,Arsenal,https://www.arsenal.com/,1886,Emirates Stadium,Emirates Stadium,60.704,https://upload.wikimedia.org/wikipedia/commons...,"Hornsey Road, London N7 7AJ, UK"
2,7,Aston Villa,https://www.avfc.co.uk/,1874,Villa Park,Villa Park,43.205,https://upload.wikimedia.org/wikipedia/commons...,"Trinity Road, Aston, Birmingham B6 6HE, UK"
3,91,Bournemouth,https://www.afcb.co.uk/,1890,Vitality Stadium,Vitality Stadium,11.307,https://upload.wikimedia.org/wikipedia/commons...,"Dean Court, Kings Park, Bournemouth BH7 7AF, UK"
4,94,Brentford,https://www.brentfordfc.com/en,1889,Gtech Community Stadium,Gtech Community Stadium,17.250,https://upload.wikimedia.org/wikipedia/commons...,"Lionel Road South, Brentford TW8 0RU, UK"
5,36,Brighton and Hove Albion,https://www.brightonandhovealbion.com/pages/en...,1901,American Express Stadium,American Express Stadium,31.876,https://upload.wikimedia.org/wikipedia/commons...,"Village Way, Brighton BN1 9BL, UK"
6,90,Burnley,https://www.burnleyfootballclub.com/,1882,Turf Moor,Turf Moor,21.944,https://upload.wikimedia.org/wikipedia/commons...,"Harry Potts Way, Burnley BB10 4BX, UK"
7,8,Chelsea,https://www.chelseafc.com/en,1905,Stamford Bridge,Stamford Bridge,41.631,https://upload.wikimedia.org/wikipedia/commons...,"Fulham Road, London SW6 1HS, UK"
8,31,Crystal Palace,https://www.cpfc.co.uk/,1861,Selhurst Park,Selhurst Park,25.456,https://upload.wikimedia.org/wikipedia/commons...,"Selhurst Road, London SE25 6PU, UK"
9,11,Everton,https://www.evertonfc.com/,1878,Hill Dickinson Stadium,Goodison Park,39.572,https://upload.wikimedia.org/wikipedia/commons...,"Goodison Road, Liverpool L4 4EL, UK"
